# **RECORDLINKAGE  TOOL**

### Required imports

In [ ]:
import pandas as pd
import numpy as np
import recordlinkage
import mlxtend
import random
import pickle
import shapash 

from shapash.explainer.smart_explainer import SmartExplainer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score
from lightgbm import LGBMClassifier
from mlxtend.plotting import plot_confusion_matrix
from recordlinkage import datasets
from codecarbon import EmissionsTracker
tracker = EmissionsTracker()



### Loading datasets 

In [ ]:
first_febrl_dataset, second_febrl_dataset = datasets.load_febrl4(return_links=False)

In [ ]:
indexes_frst_febrl_dataset = sorted(first_febrl_dataset.index)
indexes_scd_febrl_dataset = sorted(second_febrl_dataset.index)
tuples_true_links = list(zip(indexes_frst_febrl_dataset, indexes_scd_febrl_dataset))
true_links = pd.MultiIndex.from_tuples(tuples_true_links, names=('org_data', 'dpl_data'))

In [ ]:
first_febrl_dataset.head()

In [ ]:
second_febrl_dataset.head()

## **ETAPE D'INDEXING**

L'indexation n'est faisable que sur **un** ou **deux** jeu de données. Il propose 4 méthodes d'indexations:
- Full : renvoie toutes les combinaisons de paires possibles
- Block : renvoie tous les éléments qui concordent par rapport aux variables données en entrée 
- SortedNeighbourhood : renvoie tous les éléments qui concordent par rapport aux variables données en entrée et celles dans leur voisinnage
- Random : renvoi des paires crées aléatoirement

Ici, nous testons la méthode du block en utilisant les colonnes 'given_name', 'address_1' et 'date_of_birth'. Les candidats seront filtrés pour n'inclure que ceux dont les valeurs sont égales sur une ou plusieurs de ces colonnes.

L'étape d'indexation est cruciale. Plus on donne entrée des informations précises, meilleure est la selection de paires de candidats à comparer.

Pour évaluer les résultats de la méthode d'indexation, nous regardons deux élements:
- le pourcentage de paires dupliquées detctées par la méthode d'indexation utilisée
- la performance de la méthode en fonction du nombre de blocking key données en entrée


### **Méthode du block**

In [ ]:
def test_block_indexing(blocking_key: list):
    indexer = recordlinkage.Index()
    for column in blocking_key:
        indexer.block(on=column)
        pairs_to_comp = indexer.index(first_febrl_dataset, second_febrl_dataset)
        real_link_found = pairs_to_comp & true_links
        print("Nombre de paires sélectionnées: {}".format(len(pairs_to_comp)))
        print("Pourcentage de vraies paires selctionnées: {}%.".format(len(real_link_found)/len(true_links)*100))
        
    return pairs_to_comp

In [ ]:
pairs_to_comp_block = test_block_indexing(['given_name', 'address_1', 'date_of_birth'])

Plus il y a de colonnes, plus les paires sélectionnées sont réalistes et améliorent les scores de classification

**Ressemblance des paires selectionnées**

In [ ]:
def random_color():
    rgbl=['yellow','red','green']
    return random.choice(rgbl)

def selected_pairs_values(list_tuple:list):
    selected_pairs_values_df = pd.DataFrame(columns=first_febrl_dataset.columns)
    for tpl in list_tuple:
        first_candidate = first_febrl_dataset.iloc[first_febrl_dataset.index.isin([tpl[0]])]
        second_candidate = second_febrl_dataset.iloc[second_febrl_dataset.index.isin([tpl[1]])]
        concat_df = pd.concat([first_candidate, second_candidate])
        #candidates_df = candidates_df.style.applymap(lambda x: 'background-color: yellow' if True else '')
        
        selected_pairs_values_df = pd.concat([selected_pairs_values_df, concat_df])
    return selected_pairs_values_df 


In [ ]:
selected_pairs_values(pairs_to_comp_block[:4])

### **Méthode du SortedNeighborhood**

Dans le cas ou les données sont susceptible de contenir des erreurs de typographies, utiliser la méthode SortedNeighborhood peut ajouter une certaine flexibilité pour les fautes d'orthographe mineures. 

In [ ]:
def test_sorted_neighbourhood_indexing(blocking_key: list):
    for column in blocking_key:
        indexer = recordlinkage.SortedNeighbourhoodIndex(column)
        pairs_to_comp = indexer.index(first_febrl_dataset, second_febrl_dataset)
        real_link_found = pairs_to_comp & true_links
        print("Nombre de paires sélectionnées: {}".format(len(pairs_to_comp)))
        print("Pourcentage de vraies paires selctionnées: {}%.".format(len(real_link_found)/len(true_links)*100))
        
    return pairs_to_comp

In [ ]:
pairs_to_comp_sn = test_sorted_neighbourhood_indexing(['given_name', 'address_1', 'date_of_birth', 'soc_sec_id'])

La méthode basée sur le voisinnage à besoin de plus d'information pour avoir une bonne record rapport.

**COMPARAISON**

La méthode suivante consiste à comparer les paires en utilisant Compare. </br>
Nous pouvons définir plusieurs options pour la façon dont nous voulons comparer les colonnes de données.

In [ ]:
comp = recordlinkage.Compare()

# initialise similarity measurement algorithms
comp.string('given_name', 'given_name', method='jarowinkler')
comp.string('surname', 'surname', method='jarowinkler')
comp.string('address_1', 'address_1', method='levenshtein')
comp.exact('soc_sec_id', 'soc_sec_id')

# the method .compute() returns the DataFrame with the feature vectors.
comp_scores_vec = comp.compute(pairs_to_comp_block, first_febrl_dataset, second_febrl_dataset)

In [ ]:
comp_scores_vec['tupled_index'] = comp_scores_vec.index.tolist()
comp_scores_vec['label'] = comp_scores_vec['tupled_index'].apply(lambda x: 1 if x in true_links else 0)
comp_scores_vec.columns = ['given_name_score', 'surname_score', 'address_1_score', 'soc_sec_id_score', 'tupled_index', 'label']

comp_scores_vec

In [ ]:
train, test = train_test_split(comp_scores_vec, test_size=0.25)
classifier = LGBMClassifier()

In [ ]:
display(train)
Y_train = train['label']
X_train = train[['given_name_score', 'surname_score', 'address_1_score', 'soc_sec_id_score']]
Y_test = test['label']
X_test = test[['given_name_score', 'surname_score', 'address_1_score', 'soc_sec_id_score']]

In [ ]:
classifier.fit(X_train, Y_train)
predictions = classifier.predict(X_test)

Evaluation

In [ ]:
def evaluation(Y_test, predictions):
    print("Rappel: {}".format(recall_score(Y_test, predictions)))
    print("Precision: {}".format(precision_score(Y_test, predictions)))
    print("Accuracy: {}".format(accuracy_score(Y_test, predictions)))
    
    plot_confusion_matrix(confusion_matrix(Y_test, predictions))

In [ ]:
evaluation(Y_test, predictions)


### **Human review**

In [ ]:
print(len(X_train))
print(len(train))

In [ ]:
features = {
     'given_name_score': 'given_name_score',
     'surname_score': 'surname_score',
     'address_1_score': 'address_1_score',
     'soc_sec_id_score': 'soc_sec_id_score'
}
interpretable_train = train[['given_name_score', 'surname_score', 'address_1_score', 'soc_sec_id_score']]

In [ ]:
from shapash.explainer.smart_explainer import SmartExplainer
xpl = SmartExplainer(features_dict=features) # optional parameter
xpl.compile(
    x=X_test,
    model=classifier,
    y_pred= Y_test
)

In [ ]:
app = xpl.run_app(title_story='Dupliacte Matches')


In [ ]:
features = {'given_name': '',
 'surname': '',
 'street_number': '',
 'address_1': '',
 'address_2': '',
 'suburb': '',
 'postcode': '',
 'state': '',
 'date_of_birth': '',
 'soc_sec_id': ''}

In [ ]:
from shapash.explainer.smart_explainer import SmartExplainer
xpl = SmartExplainer(features_dict=features) # optional parameter
xpl.compile(
    x=train,
    model=clf,
    y_pred=predictions_monoindex['label']
)

In [ ]:
confusion_matrix = recordlinkage.confusion_matrix(test_matches_index, predictions, len(test))

In [ ]:
plot_confusion_matrix(confusion_matrix);

In [ ]:
def select_candidates_from_false_negatif_prediction():
    false_negatif_df = pd.DataFrame(columns = first_febrl_dataset.columns)
    predicted_non_match = list(set(test.index) - set(predictions))
    # format change
    predicted_non_match = pd.MultiIndex.from_tuples(predicted_non_match, names=('org', 'dup'))
    false_negatif = predicted_non_match & true_links 
        
    for tpl in false_negatif: 
        first_candidate = first_febrl_dataset.iloc[first_febrl_dataset.index.isin([tpl[0]])]
        second_candidate = second_febrl_dataset.iloc[second_febrl_dataset.index.isin([tpl[1]])]
        false_negatif_df = pd.concat([false_negatif_df, first_candidate, second_candidate])
    return false_negatif_df 

In [ ]:
select_candidates_from_false_negatif_prediction()

In [ ]:
def select_candidates_from_false_positif_prediction():
    false_positif_df = pd.DataFrame(columns = first_febrl_dataset.columns)
    false_positif = list(set(predictions) - set(true_links))
    
    for tpl in false_positif: 
        first_candidate = first_febrl_dataset.iloc[first_febrl_dataset.index.isin([tpl[0]])]
        second_candidate = second_febrl_dataset.iloc[second_febrl_dataset.index.isin([tpl[1]])]
        false_positif_df = pd.concat([false_positif_df, first_candidate, second_candidate])
    return false_positif_df 

In [ ]:
faux_positifs = select_candidates_from_false_positif_prediction()

**EVALUATION**

In [ ]:
faux_positifs.columns
faux_positifs_dict = dict(zip(faux_positifs.columns, ['']*len(faux_positifs.columns)))

In [ ]:
faux_positifs_dict

In [ ]:
print(len(predictions))
print(len(test))

In [ ]:
from shapash.explainer.smart_explainer import SmartExplainer
xpl = SmartExplainer(features_dict=faux_positifs_dict) # optional parameter
xpl.compile(
    x=train,
    model=clf,
    y_pred=predictions
)

In [ ]:
def evaluation(true_links_, pred_links):
    print("Rappel: {}".format(sklearn.recall(true_links, predictions)))
    print("Precision: {}".format(sklearn.precision(true_links, predictions)))
    print("Accuracy: {}".format(sklearn.accuracy(true_links, predictions, comp_scores_vec.index)))
    
    confusion_matrix = recordlinkage.confusion_matrix(golden_match_index_test, link_pred, len(comp_scores_vec_test))
    plot_confusion_matrix(confusion_matrix)

In [ ]:
evaluation(true_links, predictions)

### **CLASSIFICATION NON SUPERVISEE**

In [ ]:
model_km = recordlinkage.KMeansClassifier()
predict_links_km = model_km.fit_predict(comparison_vectors=comp_scores_vec)

In [ ]:
confusion_matrix_km = recordlinkage.confusion_matrix(true_links, predict_links_km, len(comp_scores_vec))
plot_confusion_matrix(confusion_matrix_km);

# Code Carbone